In [53]:
import random
import json
import numpy as np
import pickle


import nltk


from nltk.stem import WordNetLemmatizer
import tensorflow

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

lemitizer = WordNetLemmatizer()

intents = json.loads(open('C:/Users/snehp/Desktop/Python Programms/Projects/data.JSON').read())

words = []
classes = []
documents = []
ignore_letter = ['?','!','.',',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        words_list = nltk.word_tokenize(pattern)
        words.extend(words_list)
        documents.append(((words_list),intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemitizer.lemmatize(word) for word in words if word not in ignore_letter]
words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words,open('word.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

trainning = []
output_empty = [0]*len(classes)
for document in documents:
    bag = []
    word_pattern = document[0]
    word_pattern = [lemitizer.lemmatize(word.lower()) for word in word_pattern]
    for word in  words:
        if word not in word_pattern:
           bag.append(0)
        else:
           bag.append(1);
    
    output = list(output_empty)
    output[classes.index(document[1])]=1
    trainning.append([bag,output])

random.shuffle(trainning)
trainning = np.array(trainning)
train_x = list(trainning[:,0])
train_y = list(trainning[:,1])

model = Sequential(
    [ 
        Dense(128,input_shape=(len(train_x[0]),), activation = 'relu'),
        Dropout(0.5),
        Dense(64, activation = 'relu'),
        Dropout(0.5),
        Dense(len(train_y[0]), activation = 'softmax')    # < softmax activation here
    ]
)

model.compile(
    loss=tensorflow.keras.losses.CategoricalCrossentropy(),
    optimizer=SGD(learning_rate=0.01, weight_decay=1e-6, momentum=0.9, nesterov=True),
    metrics=['accuracy']
)

hist=model.fit(np.array(train_x),np.array(train_y), epochs=200, batch_size=5, verbose=0)
model.save('chatbot_model.h5',hist)
print('Done')

C:\Users\snehp\AppData\Local\Temp\ipykernel_4976\3401101219.py:58: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  trainning = np.array(trainning)


Epoch 1/200
4/4 [==============================] - 1s 5ms/step - loss: 1.2692 - accuracy: 0.5000
Epoch 2/200
4/4 [==============================] - 0s 5ms/step - loss: 1.4099 - accuracy: 0.2500
Epoch 3/200
4/4 [==============================] - 0s 5ms/step - loss: 1.3358 - accuracy: 0.4375
Epoch 4/200
4/4 [==============================] - 0s 5ms/step - loss: 1.2607 - accuracy: 0.5000
Epoch 5/200
4/4 [==============================] - 0s 3ms/step - loss: 1.2232 - accuracy: 0.5000
Epoch 6/200
4/4 [==============================] - 0s 5ms/step - loss: 1.2023 - accuracy: 0.5000
Epoch 7/200
4/4 [==============================] - 0s 5ms/step - loss: 1.2129 - accuracy: 0.5000
Epoch 8/200
4/4 [==============================] - 0s 3ms/step - loss: 1.1089 - accuracy: 0.5625
Epoch 9/200
4/4 [==============================] - 0s 5ms/step - loss: 1.0695 - accuracy: 0.6250
Epoch 10/200
4/4 [==============================] - 0s 3ms/step - loss: 0.9892 - accuracy: 0.7500
Epoch 11/200
4/4 [===========

In [56]:
import random
import json
import numpy as np
import pickle


import nltk


from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model

lemitizer = WordNetLemmatizer()

words = pickle.load(open('word.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.h5')
intents = json.loads(
    open('C:/Users/snehp/Desktop/Python Programms/data.JSON').read())


def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemitizer.lemmatize(w) for w in sentence_words]
    return sentence_words


def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = []
    for w in words:
        if w in sentence_words:
            bag.append(1)
        else:
            bag.append(0)
    return np.array(bag)


def predict(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    result = [[i, r] for i, r in enumerate(res)]
    result.sort(key=lambda x: x[1], reverse=True)
    tag = classes[result[0][0]]
    for intent in intents['intents']:
        if intent['tag'] == tag:
            print(intent['responses'][0])
            break


while True:
    print("Enter 1 for Conversation Or Enter 2 for end the conversation")
    num = int(input("Enter the number : "))
    if num == 2:
        break;
    str = input("You : ")
    print("Computer :",end=' ')
    predict(str);


    

1/1 [==============================] - 0s 132ms/step
greetings
Hello , How can i help you!
